# Finne alle emne dette semesteret
Eg leiter opp alle emne dette semesteret, og sjekke rom det er nokon emne med lærar og student ssom ikkje er publisert.

In [1]:
import requests
import os
import pandas as pd
tokenCanvas = os.environ['tokenCanvas']

headers = {
    'Authorization': f'Bearer {tokenCanvas}',
}
params = {
    'per_page': 100,
}

def finn_rel(link_header):
    link_header_dict = {}
    for link in link_header.split(","):
        url, rel = link.strip().split(";")
        rel = rel.split('=')[1]
        link_header_dict[rel.strip().replace('"', '')] = url.strip().replace('<', '').replace('>', '')      # Kommentar: denne er med BERRE for å skape balanse i .org-fila: \"
    return link_header_dict


Her hentar eg inn alle emna, og sjekker dei mot det aktuelle semesteret. Det er alle emne som har undervising før, samtidig med, eller etter "2025 HØST".

In [2]:
url = "https://hvl.instructure.com/api/v1/accounts/1/courses"
terminar = pd.read_csv("terminar.csv").to_numpy()


In [3]:
dr_liste = []
hentmeir = True
while hentmeir:
    respons = requests.get(url, headers=headers, params=params)
    if 200 <= respons.status_code < 300:
        data = respons.json()
        # print(respons.headers)
        hentmeir = "next" in respons.headers['link']
        if hentmeir:
            url = finn_rel(respons.headers['link'])['next']
            print(url)   # for kontrollen sin skuld
# Hent ut data (NB! dette vil variere frå endepunkt til endepunkt)
        dataliste = []
        for element in data:
            if element['enrollment_term_id'] in terminar:
                dataliste.append([element['id'], element['workflow_state']])
        df = pd.DataFrame(dataliste, columns=['id', 'workflow_state'])
        dr_liste.append(df)
    # hentmeir = False
# og etter at eg er ferdig å lese inn slår eg dei saman:
alledata = pd.concat((df for df in dr_liste if not df.empty), ignore_index=True)

https://hvl.instructure.com/api/v1/accounts/1/courses?page=2&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=3&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=4&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=5&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=6&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=7&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=8&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=9&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=10&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=11&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=12&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=13&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=14&per_page=100
https://hvl.instructure.com/api/v

No vil ramma *alledata* innehalde Canvas-ID for alle emne. Så kan eg gå gjennom kvart emne og hente ut litt info:

- er det nokon med "enrollment" lik "teacher"?
- status for emnet

Eg lagrer fila slik at eg slepp å hente den fram att ein gang til.

In [6]:
alledata.to_csv('alledata.csv', index=False)  # Lagre til CSV-fil for vidare bruk

## Torsdag 250821
Les inn lista frå i går:

In [2]:
data = pd.read_csv('alledata.csv')

Så kan eg sjekke kor mange emne av kvar sort:

In [3]:
data['workflow_state'].value_counts()

workflow_state
unpublished    297
available      169
Name: count, dtype: int64

Så kan eg sjekke alle emne med "unpublished"; kva informasjon kan eg få ut? Eg er mest interessert i om det er studentar og/eller lærarar der, og om emnet er krysslista til andre emne. 

Eg prøver med krysslisting først:

In [ ]:
url = "https://hvl.instructure.com/api/v1/courses/{emne_id}/sections"
temp = []
for emne_id in data[data['workflow_state'] == 'unpublished']['id']:
    print(f"Emne ID: {emne_id}")
    respons = requests.get(url.format(emne_id=emne_id), headers=headers)
    if 200 <= respons.status_code < 300:
        svar = respons.json()
        if isinstance(svar, list):
            if len(svar) > 0:
                for section in svar:
                    temp.append(emne_id)
                    print(f"  Section ID: {section['id']}, Name: {section['name']}")
            else:
                print(f"  No sections found")

aktuelle_emne = list(set(temp))          

Emne ID: 18614
  Section ID: 24924, Name: PHD950-1 22V
Emne ID: 22659
  Section ID: 30992, Name: PHD950-1 22H
Emne ID: 22664
  Section ID: 30994, Name: PHD950-1 23V
Emne ID: 25943
  No sections found
Emne ID: 26608
  Section ID: 35589, Name: PHD950-1 23H
Emne ID: 26612
  Section ID: 35594, Name: PHD950-1 24V
Emne ID: 29030
  Section ID: 39774, Name: GE4-304-1 25V
Emne ID: 29108
  No sections found
Emne ID: 29119
  No sections found
Emne ID: 29139
  No sections found
Emne ID: 29149
  No sections found
Emne ID: 29153
  Section ID: 40044, Name: MACREL-OPG-1 25V
Emne ID: 29160
  No sections found
Emne ID: 29199
  No sections found
Emne ID: 29236
  Section ID: 39723, Name: KASP540-1 25V
Emne ID: 29412
  Section ID: 39882, Name: LEI116-1 25V
Emne ID: 29439
  Section ID: 39911, Name: LEI352-1 25V
Emne ID: 29479
  No sections found
Emne ID: 29481
  Section ID: 40048, Name: MDIP550-1 25V
Emne ID: 29484
  Section ID: 39856, Name: ING5002D-1 25V
Emne ID: 29486
  Section ID: 40050, Name: MDIPD550-

Då har eg ein fin metode for å sjekke om eit emne har krysslista vekk seksjonane sine. Dei emna treng eg i alle fall ikkje sjekke. Så kan eg gå gjennom alle dei aktuelle og finne omikkje anna så i alle fall namnet på emnet og alle lærarane:

In [36]:
emneliste = [33899, 31232]
emneliste = aktuelle_emne
upubliserte_emne = []
for emne_id in emneliste:
    url = "https://hvl.instructure.com/api/v1/courses/{emne_id}/"
    params = {
        'include[]': ['teachers', 'total_students'],
    }
    respons = requests.get(url.format(emne_id=emne_id), headers=headers, params =params)
    if 200 <= respons.status_code < 300:
        svar = respons.json()
        print(svar)
        upubliserte_emne.append({
            'id': emne_id,
            'name': svar['name'],
            'teachers': [teacher['display_name'] for teacher in svar['teachers']],
            'total_students': svar['total_students']
        })
    else:
        print(f"Error fetching course {emne_id}: {respons.status_code}")
    

{'id': 31232, 'name': 'BBLDPR201 Bergen (1)', 'account_id': 18196, 'uuid': 'HylRMxQsLbfezheQl6NkoB5Pc1ouVgE4HPxAy9KT', 'start_at': None, 'grading_standard_id': None, 'is_public': None, 'created_at': '2025-04-01T07:18:08Z', 'course_code': 'BBLDPR201 Bergen (1)', 'default_view': 'modules', 'root_account_id': 1, 'enrollment_term_id': 352, 'license': None, 'grade_passback_setting': None, 'end_at': None, 'public_syllabus': False, 'public_syllabus_to_auth': False, 'storage_quota_mb': 524, 'is_public_to_auth_users': False, 'homeroom_course': False, 'course_color': None, 'friendly_name': None, 'hide_final_grades': False, 'apply_assignment_group_weights': False, 'total_students': 19, 'teachers': [{'id': 54859, 'anonymous_id': '16bv', 'display_name': 'Randi Tiedemann Gausemel', 'avatar_image_url': 'https://hvl.instructure.com/images/thumbnails/1150533/vnqRNQfBf3gf7ZqGRyyGA0mOnxJhVUHjO3jd50vo', 'html_url': 'https://hvl.instructure.com/users/54859', 'pronouns': None}], 'calendar': {'ics': 'https:/

No har eg ei relativt komplett liste; denne kan eg eksportere til CSV og bearbeide vidare i Excel. (Det gjorde eg DataWrangler)